In [ ]:
from unet import *
from res_unet import *
from data import *
import os
import h5py
from keras.utils import plot_model

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [ ]:
input_shape = (128,128)
# model = unet(batchnorm = False, input_size=input_shape+(1,))
model = res_unet(batchnorm = False, input_size=input_shape+(1,))
model.summary()

In [ ]:
model_name = 'res_unet_bus_base_w_normals_aug'
# plot_model(model, to_file='model_architectures/{}.png'.format(model_name))

## Train your Unet with bus data

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

In [ ]:
#Training Data
train_image_path = 'data/seg_dir/train_images'
train_mask_path = 'data/seg_dir/train_masks'
len_training_set = len(os.listdir(train_image_path))
print('len training set -->', len_training_set)

#Validation Data
val_image_path = 'data/seg_dir/val_images'
val_mask_path = 'data/seg_dir/val_masks'
len_val_set = len(os.listdir(val_image_path))
print('len validation set -->', len_val_set)

### Train with npy file

In [ ]:
aug = 'on'

imgs_train,imgs_mask_train = train_generator_npy(input_shape, train_image_path, train_mask_path,'','')
imgs_val,imgs_mask_val = train_generator_npy(input_shape, val_image_path, val_mask_path,'','')
seed = 1

#Training Data
train_batch_size = 16

data_gen_args = dict(width_shift_range=0.1,
                    height_shift_range=0.1,
                    shear_range=10,
                    horizontal_flip=True,
                    fill_mode='nearest')

if aug == 'on':
    train_image_datagen = ImageDataGenerator(**data_gen_args)
    train_mask_datagen = ImageDataGenerator(**data_gen_args)
else:
    train_image_datagen = ImageDataGenerator()
    train_mask_datagen = ImageDataGenerator()

train_image_generator = train_image_datagen.flow(imgs_train, batch_size=train_batch_size, seed=seed) #save_to_dir='/external_drive/BUS_Deep_Learning/data/seg_dir_nn_unet/train_aug_images'
train_mask_generator = train_mask_datagen.flow(imgs_mask_train, batch_size=train_batch_size, seed=seed) #save_to_dir='/external_drive/BUS_Deep_Learning/data/seg_dir_nn_unet/train_aug_masks'

tsg = zip(train_image_generator, train_mask_generator)

#Validation Data

val_batch_size = 16

val_image_datagen = ImageDataGenerator()
val_mask_datagen = ImageDataGenerator()

val_image_generator = train_image_datagen.flow(imgs_val, batch_size=val_batch_size, seed=seed)
val_mask_generator = train_mask_datagen.flow(imgs_mask_val, batch_size=val_batch_size, seed=seed)

vsg = zip(val_image_generator, val_mask_generator)

# # Final Generators
# tsg = convert_mask_generator(tsg)
# vsg = convert_mask_generator(vsg)

In [ ]:
model_dir = 'trained_models'
model_checkpoint = ModelCheckpoint(model_dir+'/'+model_name+'.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
history = model.fit_generator(tsg, validation_data = vsg, steps_per_epoch=len_training_set/16, validation_steps=len_val_set/16,
                    epochs=50,callbacks=[model_checkpoint])

In [ ]:
with h5py.File(model_dir+'/'+'{}_history.hdf5'.format(model_name)) as hfile:
    for keyi in history.history:
        hfile.create_dataset(keyi, data=history.history[keyi])